## 1-minute introduction to Jupyter ##

A Jupyter notebook consists of cells. Each cell contains either text or code.

A text cell will not have any text to the left of the cell. A code cell has `In [ ]:` to the left of the cell.

If the cell contains code, you can edit it. Press <kbd>Enter</kbd> to edit the selected cell. While editing the code, press <kbd>Enter</kbd> to create a new line, or <kbd>Shift</kbd>+<kbd>Enter</kbd> to run the code. If you are not editing the code, select a cell and press <kbd>Ctrl</kbd>+<kbd>Enter</kbd> to run the code.

---

# Lesson 10c: Handling errors with `try-except`

Now that you have learnt how to test your programs with `assert`, let’s look at handling errors.

When you are using a program, it is esepcially annoying when the program crashes and exits. You curse and swear at the company that released this program, wondering if they actually know what they are doing. Crashing all the way back to your operating system is *catastrophic*.

You would usually expect the program to handle errors, at the very least telling you that an error occured, displaying some helpful info about it, and maybe asking you if you want to submit a bug report so they can fix it.

### When to catch errors

In Python, we don’t always expect errors to happen. For instance, when we use the `find()` string method, we know that it will return `-1` if nothing is found, so we don’t need to catch errors there.

In other cases, such as when opening a file, all kinds of things can happen.

In [ ]:
filename = 'data.txt'
f = open(filename, 'r')

The file might not exist, or it might be in a wrong format, or it might not be a valid text file, or … we definitely expect that some kind of error is going to come up. This is a good place to use `try`.


## The `except` companion keyword

The `try` keyword tells Python that the indented chunk of code might `raise` errors. The `except` keyword is the companion to try, just like `else` is a companion to `if`. Any errors named after `except` will be caught be `except` and handled by its indented code.

Let’s see how it works:

In [ ]:
filename = 'data.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print('File was not found.')

Notice that `FileNotFoundError` did not cause the program to exit prematurely, unlike the previous cell. The `except` keyword caught the `FileNotFoundError` and ran the indented code cell, which printed the error.

So how do we tell Python to continue with our code if `FileNotFoundError` _was not raised_?

In [ ]:
filename = 'data.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print('File was not found.')
else:
    data = f.readlines()

`try-except` also accepts an `else` keyword. Code indented under `else` will run **only if the `try` statement was successful**.

We learnt in Lesson 5b that file handles must always be closed properly. Where should we put `f.close()`?

If we indent it under `except`, it will only close the file when `FileNotFoundError` is raised.

If we indent it under `else`, it will only close the file when no error is raised.

If we wnt `f.close()` to be called **regardless of the outcome of `try`**, we do this:

In [ ]:
filename = 'data.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print('File was not found.')
else:
    data = f.readlines()
finally:
    f.close()

Code that is indented under the `finally` keyword will run regardless of the outcome of `try`.

## Catching generic errors

In the above code, if we got some other error besides `FileNotFoundError`, it would have slipped past our `try-except` statement.

Just as we have `if-elif-else` statements, we often also have a final `except` to catch all other uncaught errors. Besides `FileNotFoundError`, who knows what other errors we might get? This is what we would do:

In [ ]:
filename = 'data.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print(f'File \'{filename}\' was not found.')
except:
    print('An error occured.')
else:
    data = f.readlines()
finally:
    f.close()

Hmm, but that’s not very helpful. If you were debugging and you simply saw `An error occured`, you would have no idea where to begin debugging (besides knowing that the error was not one of those you were looking out for).

We can try to catch generic errors and print them like this:

In [ ]:
filename = 'data.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print(f'File \'{filename}\' was not found.')
except Exception as e:
    print(e)
else:
    data = f.readlines()
finally:
    f.close()

Here, `Exception` is an object that represents a generic error (This will make more sense after Lesson 9 and 10, on Object-Oriented Programming).

`except Exception` tells Python to catch any exceptions (if it is not a `FileNotFoundError` which would have been caught earlier), and run the indented code.

`except Exception as e` tells Python to catch any exceptions and assign them to the variable `e`. Exceptions are objects in Python too, with attributes and methods. We can then do things with `e`, such as print it for a more helpful statement.

## Capturing errors as variables

Lets see an example where this would be helpful:

In [ ]:
filename = 'emoji.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print(f'File \'{filename}\' was not found.')
except Exception as e:
    print(e)
else:
    data = f.readlines()
finally:
    f.close()

Oops … we encountered an error on line 9, within the `else` statement. Guess we need to nest another `try-except` there.

If we just use a simple `try-except` without assigning the exception:

In [ ]:
filename = 'emoji.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print(f'File \'{filename}\' was not found.')
except Exception as e:
    print(e)
else:
    # Let’s nest another try-except to catch file read errors
    try:
        data = f.readlines()
    except:
        print('File read error')
finally:
    f.close()

We get an unhelpful error message. If we want to examine what kind of error it is, we catch the generic exception (through the `Exception` object), and assign it to a variable (I usually just use `e`):

In [ ]:
filename = 'emoji.txt'
try:
    f = open(filename, 'r', encoding="ascii")
except FileNotFoundError:
    print(f'File \'{filename}\' was not found.')
except Exception as e:
    print(e)
else:
    try:
        data = f.readlines()
    # Catch all exceptions and assign them to e
    except Exception as e:
        print(e)
finally:
    f.close()

No more `UnicodeDecodeError`, instead we get the error message that it would have printed.

From this, we can tell that some characters in `'emoji.txt'` are not in the ASCII range. Well, all emoji characters are not in the ASCII table, but we haven’t learnt this yet.

## Other applications of `try-except`

Let’s see where else we can use this.

Some of you have discovered a quick way to check if a number is float:

In [ ]:
def isfloat(num_str):
    try:
        float(num_str)
    except ValueError:
        return False
    else:
        return True
        
isfloat('1.234.5')

This works by simply trying to cast the number to a `float`. Normally, a `ValueError` exception will be raised if the number is not a valid `float`. We can catch that exception, knowing that this means the number cannot be cast to a `float`, and use that to return `False`.

Note that we can do this witn `int()` too!

Other examples come when you get deeper into Python. There are many commands that naturally raise `Exception`s, many of them from opening files or network connections. We will look at those when we encounter them.

## `try-except` vs `if-else`

These two seem to be somewhat similar: Both run different sets of code depending on the outcome of a line (or a few lines of code).

The difference is that `if-else` checks for a (boolean) result of an expression, `try-except` checks for `Exception`s.

It is important that you do not get into the bad habit of using `try-except` to catch errors that should be fixed by proper programming logic. For example, catching `ZeroDivisionError` when you should be validating your input properly.

`if-else` should be used whenever it is possible to check that certain conditions are met.

`try-except` should **only** be used:

- when `Exception`s are **unavoidable in the course of running the program**, and need to be handled properly,
- if it provides a much faster/simpler alternative to writing your own (possibly buggy) code (for example, catching `ValueError` from `float()` to determine if a number is a `float`).

## Good `try-except` practices

### Minimise `try` code

A good practice when using `try-except` is to write **as little code as possible** within the `try` statement.

Since the `Exception` is handled by `except` and does not reach the Python debugger, you will have _no information_ about which line is responsible for raising the `Exception`. this makes your debugging harder compared to `if-else`. Keeping the `try` code minimal makes it clear what is causing the problem.

### Re-raising uncaught `Exception`s

When in doubt, it is always a good idea to put the following line of code as the last `except` statement:

    except:
        raise

The `raise` statement, when used like this without any exceptions named after it, will **re-raise the last `Exception`**. This is very handy for debugging, as any `Exception`s that were not caught will still be raised, giving you info about which line of code caused it and what the problem is.

### Closing resources properly in `finally`

If the code you are writing requires resources to be closed/handled properly, it is good practice to do so in the `finally` statement.

Closing file handles, database cursors, network connections, etc should be done here.